In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", sep = "\t")
#train2 = pd.read_csv("../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip", "\t")
print(train.head())
#the shape is 25000 * 3

In [6]:
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip",sep = "\t")

In [7]:
print(train.loc[0]["review"])
#有许多html符号, 要将他去掉

In [8]:
from bs4 import BeautifulSoup
#"html.parse" 是固定参数,不要忘记写
example = BeautifulSoup(train.loc[0]["review"], "html.parser")
print(example.get_text())
import re
p = re.compile(r'[^a-zA-A]')

In [9]:
print(type(p))
#re.patern
print(type(p.sub(" ", example.get_text())))
letters_only = re.sub('[^A-Za-z]', ' ', example.get_text())
print(type(letters_only))
#str
words = letters_only.split()
print(words)

In [10]:
import re
p = re.compile(r'[^a-zA-A]')
reviews = []

from nltk.corpus import stopwords
from nltk.corpus import stopwords
stops = set( stopwords.words('english'));
#print(stops)

for i in train.loc[:]["review"] :
    #去掉html,只取文本
    finished_row_word = []
    sub_html = BeautifulSoup(i, "html.parser").get_text();
    #符号替换为空格
    sub_space_html = p.sub(" ", sub_html);
    #print(pure)
    pure_words = sub_space_html.lower().split();

    for word in pure_words:
        if word not in stops:
            #print(word)
            finished_row_word.append(word);
            
            
    #语法：  'sep'.join(seq)
    #参数说明
    #sep：分隔符。可以为空
    #seq：要连接的元素序列、字符串、元组、字典
    #上面的语法即：以sep作为分隔符，将seq所有的元素合并成一个新的字符串
    finished_str = " ".join(finished_row_word)
    reviews.append(finished_str)
            
    
print("clean the reviews success")
    

In [11]:

test_reviews = []

from nltk.corpus import stopwords
from nltk.corpus import stopwords
stops = set( stopwords.words('english'));
#print(stops)

for i in test.loc[:]["review"] :
    #去掉html,只取文本
    finished_row_word = []
    sub_html = BeautifulSoup(i, "html.parser").get_text();
    #符号替换为空格
    sub_space_html = p.sub(" ", sub_html);
    #print(pure)
    pure_words = sub_space_html.lower().split();

    for word in pure_words:
        if word not in stops:
            #print(word)
            finished_row_word.append(word);
            
            
    #语法：  'sep'.join(seq)
    #参数说明
    #sep：分隔符。可以为空
    #seq：要连接的元素序列、字符串、元组、字典
    #上面的语法即：以sep作为分隔符，将seq所有的元素合并成一个新的字符串
    finished_str = " ".join(finished_row_word)
    test_reviews.append(finished_str)
            
    
print("clean the test reviews success")

In [12]:
a = pd.Series(reviews)
a.head()
a.to_csv("reviews.csv",index = False)

In [18]:
# 从sklearn.feature_extraction.text里导入CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)
train_data_features = vectorizer.fit_transform(reviews)
train_data_features = pd.DataFrame(train_data_features.toarray())

test_data_features = vectorizer.transform(test_reviews)
test_data_features = test_data_features.toarray()

In [20]:
from sklearn.ensemble import RandomForestClassifier #集成学习中的随机森林
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["sentiment"] )

result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "submission.csv", index=False, quoting=3 )

In [ ]:
#Tokenizer用来对文本中的词进行统计计数，生成文档词典，以支持基于词典位序生成文本的向量表示。
from keras.preprocessing.text import Tokenizer

#使用一系列文档来生成token词典，texts为list类，每个元素为一个文档。
tokenizer.fit_on_texts(reviews)

#将多个文档转换为word下标的向量形式,shape为[len(texts)，len(text)] -- (文档数，每条文档的长度)
#texts_to_sequences(texts) 

#将多个文档转换为矩阵表示,shape为[len(texts),num_words]
tokenized_train = tokenizer.texts_to_sequences(reviews)

from keras.preprocessing.sequence import pad_sequences
max_features = 6000

tokenizer = Tokenizer(num_words=max_features)


#tokenized_train[0]



In [ ]:
maxlen = 400
X_train = pad_sequences(tokenized_train, maxlen=maxlen)
X_train.shape

In [ ]:
from collections import defaultdict
dedict = defaultdict(int)
print(dedict)

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils.np_utils import to_categorical

inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True, name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.2)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
print("successful")

In [ ]:
train['sentiment'] = to_categorical(train['sentiment'], num_classes=2)
batch_size = 32
epochs = 5
model.fit(X_train, train['sentiment'], batch_size=batch_size, epochs=epochs, validation_split=0.2)